## **1.0. Data Pre-Processing**

### 1.1. Imports

In [1]:
import pandas as pd
import random,time,csv
import numpy as np
import math,copy,os

In [2]:
MEPS16 = pd.read_csv('/kaggle/input/medical-expediture-meps-1-2/h192.csv')

# ## Drop NULL values
MEPS16 = MEPS16.dropna()


MEPS16 = MEPS16.rename(columns = {'FTSTU53X' : 'FTSTU', 'ACTDTY53' : 'ACTDTY', 'HONRDC53' : 'HONRDC', 'RTHLTH53' : 'RTHLTH',
                              'MNHLTH53' : 'MNHLTH', 'CHBRON53' : 'CHBRON', 'JTPAIN53' : 'JTPAIN', 'PREGNT53' : 'PREGNT',
                              'WLKLIM53' : 'WLKLIM', 'ACTLIM53' : 'ACTLIM', 'SOCLIM53' : 'SOCLIM', 'COGLIM53' : 'COGLIM',
                              'EMPST53' : 'EMPST', 'REGION53' : 'REGION', 'MARRY53X' : 'MARRY', 'AGE53X' : 'AGE',
                              'POVCAT16' : 'POVCAT', 'INSCOV16' : 'INSCOV'})


MEPS16 = MEPS16[MEPS16['PANEL'] == 21]
MEPS16 = MEPS16[MEPS16['REGION'] >= 0] # remove values -1
MEPS16 = MEPS16[MEPS16['AGE'] >= 0] # remove values -1
MEPS16 = MEPS16[MEPS16['MARRY'] >= 0] # remove values -1, -7, -8, -9
MEPS16 = MEPS16[MEPS16['ASTHDX'] >= 0] # remove values -1, -7, -8, -9
MEPS16 = MEPS16[(MEPS16[['FTSTU','ACTDTY','HONRDC','RTHLTH','MNHLTH','HIBPDX','CHDDX','ANGIDX','EDUCYR','HIDEG',
                         'MIDX','OHRTDX','STRKDX','EMPHDX','CHBRON','CHOLDX','CANCERDX','DIABDX',
                         'JTPAIN','ARTHDX','ARTHTYPE','ASTHDX','ADHDADDX','PREGNT','WLKLIM',
                         'ACTLIM','SOCLIM','COGLIM','DFHEAR42','DFSEE42','ADSMOK42',
                         'PHQ242','EMPST','POVCAT','INSCOV']] >= -1).all(1)]

# ## Change symbolics to numerics
MEPS16['RACEV2X'] = np.where((MEPS16['HISPANX'] == 2 ) & (MEPS16['RACEV2X'] == 1), 1, MEPS16['RACEV2X'])
MEPS16['RACEV2X'] = np.where(MEPS16['RACEV2X'] != 1 , 0, MEPS16['RACEV2X'])
MEPS16 = MEPS16.rename(columns={"RACEV2X" : "RACE"})
# MEPS16['UTILIZATION'] = np.where(MEPS16['UTILIZATION'] >= 10, 1, 0)



def utilization(row):
        return row['OBTOTV16'] + row['OPTOTV16'] + row['ERTOT16'] + row['IPNGTD16'] + row['HHTOTD16']

MEPS16['TOTEXP16'] = MEPS16.apply(lambda row: utilization(row), axis=1)
lessE = MEPS16['TOTEXP16'] < 10.0
MEPS16.loc[lessE,'TOTEXP16'] = 0.0
moreE = MEPS16['TOTEXP16'] >= 10.0
MEPS16.loc[moreE,'TOTEXP16'] = 1.0

MEPS16 = MEPS16.rename(columns = {'TOTEXP16' : 'UTILIZATION'})

MEPS16 = MEPS16[['REGION','AGE','SEX','RACE','MARRY',
                                 'FTSTU','ACTDTY','HONRDC','RTHLTH','MNHLTH','HIBPDX','CHDDX','ANGIDX',
                                 'MIDX','OHRTDX','STRKDX','EMPHDX','CHBRON','CHOLDX','CANCERDX','DIABDX',
                                 'JTPAIN','ARTHDX','ARTHTYPE','ASTHDX','ADHDADDX','PREGNT','WLKLIM',
                                 'ACTLIM','SOCLIM','COGLIM','DFHEAR42','DFSEE42', 'ADSMOK42',
                                 'PCS42','MCS42','K6SUM42','PHQ242','EMPST','POVCAT','INSCOV','UTILIZATION', 'PERWT16F']]

MEPS16 = MEPS16.rename(columns={"UTILIZATION": "Probability","RACE" : "race"})

### 2 Train-Test Splits and Data Preparation for SMOTE

In [3]:
# Import necessary libraries
from sklearn.model_selection import train_test_split

# A helper function to split data into train and test sets
def split_data(df, target, test_size=0.3, random_state=42, stratify=True):
    if target not in df.columns:
        raise ValueError(f"Target column '{target}' not found in DataFrame.")
    X = df.drop(columns=[target])
    y = df[target]
    if stratify:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, stratify=y
        )
    else:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state
        )
    return X_train, X_test, y_train, y_test

# MEPS16 Dataset
X_train_meps16, X_test_meps16, y_train_meps16, y_test_meps16 = split_data(MEPS16, target="Probability")

In [4]:
!pip install transformers datasets scikit-learn torch

In [5]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [6]:
# -------------------------------------
# 1. Data Preparation
# -------------------------------------

# We will combine X and y back into a single DataFrame for convenience.

train_meps16 = X_train_meps16.copy()
train_meps16["Probability"] = y_train_meps16

test_meps16 = X_test_meps16.copy()
test_meps16["Probability"] = y_test_meps16

# Create textual representations. For instance, describe each data point:
def row_to_text(row):
    # Here we convert numeric/categorical features into a textual description.
    
    attributes = []
    for col in row.index:
        if col != "Probability":
            attributes.append(f"{col}={row[col]}")
    description = " ".join(attributes)
    return f"This individual has the following characteristics: {description}."

# Assuming you have train_meps16 and test_meps16 DataFrames
train_meps16["text"] = train_meps16.apply(row_to_text, axis=1)
test_meps16["text"] = test_meps16.apply(row_to_text, axis=1)

# Ensure labels are numeric
train_meps16["Probability"] = train_meps16["Probability"].astype(int)
test_meps16["Probability"] = test_meps16["Probability"].astype(int)

# Convert pandas DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_meps16[["text", "Probability"]])
test_dataset = Dataset.from_pandas(test_meps16[["text", "Probability"]])

In [7]:
# -------------------------------------
# 2. Tokenization
# -------------------------------------

model_name = "bert-base-uncased"  # You can choose another model if desired
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(
        examples["text"], padding="max_length", truncation=True, max_length=128
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Rename Probability column to labels
train_dataset = train_dataset.rename_column("Probability", "labels")
test_dataset = test_dataset.rename_column("Probability", "labels")

# Remove unnecessary columns
train_dataset = train_dataset.remove_columns(["text", "__index_level_0__"])
test_dataset = test_dataset.remove_columns(["text", "__index_level_0__"])

# Set the format for PyTorch
train_dataset = train_dataset.with_format("torch")
test_dataset = test_dataset.with_format("torch")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/10971 [00:00<?, ? examples/s]

Map:   0%|          | 0/4702 [00:00<?, ? examples/s]

In [8]:
# -------------------------------------
# 3. Load Pre-trained Model
# -------------------------------------

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# -------------------------------------
# 4. Define Metrics
# -------------------------------------

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1).numpy()
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

In [10]:
# -------------------------------------
# 5. Training Arguments and Trainer
# -------------------------------------

training_args = TrainingArguments(
    output_dir="WANDB-meps16-BASE-50EP",
    evaluation_strategy="epoch",  # Ensure evaluation happens after each epoch
    save_strategy="epoch",        # Match save strategy to evaluation strategy
    save_total_limit=3,          # Keep only the latest checkpoint (final epoch)
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=50,
    weight_decay=0.01,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True  # Ensure the metric comparison works correctly
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_23/3183955806.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("wandb_api_key") 

wandb.login(key=my_secret)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [12]:
# -------------------------------------
# 6. Train the Model
# -------------------------------------

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: arman-dogru. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241220_164307-5dm6dafn
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run WANDB-meps16-BASE-50EP
wandb: ⭐️ View project at https://wandb.ai/arman-dogru/huggingface
wandb: 🚀 View run at https://wandb.ai/arman-dogru/huggingface/runs/5dm6dafn


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.451500,0.432104,0.832412,0.000000,0.000000,0.000000
2,0.394900,0.399446,0.832412,0.000000,0.000000,0.000000
3,0.416000,0.383596,0.843046,0.591912,0.204315,0.303774
4,0.358600,0.393628,0.843046,0.567204,0.267766,0.363793
5,0.343300,0.383057,0.841557,0.551807,0.290609,0.380715
6,0.330400,0.382514,0.837303,0.575163,0.111675,0.187035
7,0.376000,0.383452,0.827733,0.484973,0.450508,0.467105
8,0.417900,0.376164,0.843684,0.566751,0.285533,0.379747
9,0.479300,0.375938,0.839643,0.575893,0.163706,0.254941
10,0.416100,0.393316,0.833475,0.551020,0.034264,0.064516


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=34300, training_loss=0.3619536130734157, metrics={'train_runtime': 7799.6714, 'train_samples_per_second': 70.33, 'train_steps_per_second': 4.398, 'total_flos': 3.6082392354432e+16, 'train_loss': 0.3619536130734157, 'epoch': 50.0})

In [13]:
# -------------------------------------
# 7. Evaluate the Model
# -------------------------------------

results = trainer.evaluate()
print("Evaluation results:", results)

Evaluation results: {'eval_loss': 0.38345247507095337, 'eval_accuracy': 0.8277328796256912, 'eval_precision': 0.4849726775956284, 'eval_recall': 0.45050761421319796, 'eval_f1': 0.46710526315789475, 'eval_runtime': 16.5653, 'eval_samples_per_second': 283.847, 'eval_steps_per_second': 17.748, 'epoch': 50.0}


In [14]:
# -------------------------------------
# 9. Save the Model
# -------------------------------------

model.save_pretrained("./fine_tuned_model_meps16")
tokenizer.save_pretrained("./fine_tuned_model_meps16")

('./fine_tuned_model_meps16/tokenizer_config.json',
 './fine_tuned_model_meps16/special_tokens_map.json',
 './fine_tuned_model_meps16/vocab.txt',
 './fine_tuned_model_meps16/added_tokens.json',
 './fine_tuned_model_meps16/tokenizer.json')